In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
import tqdm
import shutil
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [ ]:
train_path = '../input/dogs-vs-cats/train.zip'
test_path = '../input/dogs-vs-cats/test1.zip'

# save all files to kaggle/files/images
destination = '/kaggle/files/images'

from zipfile import ZipFile as zipper
with zipper(train_path, 'r') as zipp:
    zipp.extractall(destination)
    
with zipper(test_path, 'r') as zipp:
    zipp.extractall(destination)

In [ ]:
train = pd.DataFrame({'file': os.listdir('/kaggle/files/images/train')})

categories = []
for i in os.listdir('/kaggle/files/images/train'):
    if 'dog' in i:
        categories.append(1)
    else:
        categories.append(0)
        
train['categories'] = categories
train['categories'] = train['categories'].replace({0: 'cat', 1: 'dog'})
train_set, val_set = train_test_split(train, test_size=0.2, random_state = 0)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.utils import to_categorical

train_gen_scaled = ImageDataGenerator(rescale=1./255)
val_gen_scaled = ImageDataGenerator(rescale=1./255)

batch_size = 64

# generating from data frame 
train_generator_scaled = train_gen_scaled.flow_from_dataframe(
    dataframe = train_set,
    directory = destination + '/train/', # file path format
    x_col = 'file',
    y_col = 'categories',
    class_mode = 'categorical',
    target_size = (224,224),
    batch_size = 64
)

validation_generator_scaled = val_gen_scaled.flow_from_dataframe(
    dataframe = val_set,
    directory = destination + '/train/',
    x_col = 'file',
    y_col = 'categories',
    class_mode = 'categorical',
    target_size = (224,224),
    batch_size = batch_size,
    shuffle = False )# ensure the validation and prediction are not random)

def mini_batch_example_plot(df):
    '''
    This function creates 8 examples plots.
    
    df - the dataframe containing examples 
    '''
    # create an another generator for training set
    example_generator = train_gen_scaled.flow_from_dataframe(
        dataframe = df,
        directory = destination + '/train/',
        x_col = 'file',
        y_col = 'categories',
        class_mode = 'categorical',
        target_size = (224,224)
    )
    
    fig, ax  = plt.subplots(2,4,figsize=(12, 12))
    ax = ax.flatten()
    
    for i in range(8):
        X, Y = next(example_generator)
        image = X[0]
        ax[i].imshow(image)
        ax[i].axis('off')
    
    
mini_batch_example_plot(train_set)
pre_trained_model = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights=None,
    input_shape=(224,224,3),
    
)

pre_trained_model.summary()

In [ ]:
from tensorflow.keras import Model
last_layer = pre_trained_model.get_layer('conv_7b_ac')
last_output = last_layer.output
model=keras
model=layers.Flatten()(last_output)
model=layers.Dense(1024,activation='relu')(model)
model=layers.Dropout(0.2)(model)
model=layers.Dense (2,activation='softmax')(model)
model = Model(pre_trained_model.input, model)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit_generator(
    train_generator_scaled,
    epochs=100,
   validation_data = validation_generator_scaled,
     validation_steps = 64,
    steps_per_epoch=64
)